# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import google_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Load city data into DataFrame from WeatherPy
cityweather_df = "../WeatherPy/Output/cityweather.csv"

# Convert 
cityweather_df = pd.read_csv(cityweather_df)

cityweather_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Jamestown,100,US,2022-02-12 05:29:42,69,42.0970,-79.2353,40.08,11.50
1,Sol'vychegodsk,100,RU,2022-02-12 05:29:42,97,61.3305,46.9156,16.32,4.29
2,Aswān,0,EG,2022-02-12 05:29:43,72,24.0934,32.9070,51.12,11.03
3,Saskylakh,76,RU,2022-02-12 05:27:23,82,71.9167,114.0833,1.90,12.97
4,Lagoa,95,PT,2022-02-12 05:29:43,80,39.0500,-27.9833,59.29,17.56
...,...,...,...,...,...,...,...,...,...
545,Chengzhong,100,CN,2022-02-12 05:33:50,80,30.9500,113.5500,44.65,8.81
546,Pacifica,20,US,2022-02-12 05:33:13,65,37.6138,-122.4869,64.83,10.36
547,Ballina,75,AU,2022-02-12 05:33:51,75,-28.8667,153.5667,73.44,12.66
548,Nizwá,0,OM,2022-02-12 05:33:51,38,22.9333,57.5333,70.30,3.33


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [46]:
# Configure gmaps with API key
gmaps.configure(api_key = google_key)

# Store latitude and longitude into locations
locations = cityweather_df[["Latitude", "Longitude"]]

# Store humidity
humidity = cityweather_df['Humidity']

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

# Create figure with layout specifications
fig = gmaps.figure(layout = figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
# A max temperature lower than 80 degrees but higher than 70.
weatherconditions_df = cityweather_df[(cityweather_df['Max Temperature'] > 70)]
weatherconditions_df =weatherconditions_df[(weatherconditions_df['Max Temperature'] < 80)]
# Zero cloudiness
weatherconditions_df = weatherconditions_df[(weatherconditions_df['Cloudiness'] == 0)]
# Wind speed less than 10 mph.
weatherconditions_df = weatherconditions_df[(weatherconditions_df['Wind Speed'] <= 10)]

weatherconditions_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
102,Ormara,0,PK,2022-02-12 05:30:29,51,25.2088,64.6357,71.78,8.72
161,Sur,0,OM,2022-02-12 05:30:58,42,22.5667,59.5289,73.42,4.36
224,Tura,0,IN,2022-02-12 05:31:27,33,25.5198,90.2201,71.44,4.43
241,Pedernales,0,DO,2022-02-12 05:31:35,65,18.0384,-71.7440,76.05,5.99
299,Kruisfontein,0,ZA,2022-02-12 05:32:02,49,-34.0033,24.7314,72.14,4.29
384,Plettenberg Bay,0,ZA,2022-02-12 05:32:40,61,-34.0527,23.3716,71.04,4.07
427,La Palma,0,US,2022-02-12 05:32:59,46,33.8464,-118.0467,77.67,3.44
447,Chabahar,0,IR,2022-02-12 05:33:08,59,25.2919,60.6430,70.90,5.59
485,Dwārka,0,IN,2022-02-12 05:33:26,38,22.2394,68.9678,75.45,5.88
490,Keti Bandar,0,PK,2022-02-12 05:33:28,37,24.1447,67.4497,75.51,8.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [89]:
# Data frame with necessary columns
hotel_df = weatherconditions_df.loc[:, ['City', 'Country', 'Latitude', 'Longitude']]

# Add empty column to data frame to store hotel names
hotel_df['Hotel Name'] = ""

# Reset index
hotel_df = hotel_df.reset_index()

hotel_df

,index,City,Country,Latitude,Longitude,Hotel Name
0,102,Ormara,PK,25.2088,64.6357,
1,161,Sur,OM,22.5667,59.5289,
2,224,Tura,IN,25.5198,90.2201,
3,241,Pedernales,DO,18.0384,-71.7440,
4,299,Kruisfontein,ZA,-34.0033,24.7314,
5,384,Plettenberg Bay,ZA,-34.0527,23.3716,
6,427,La Palma,US,33.8464,-118.0467,
7,447,Chabahar,IR,25.2919,60.6430,
8,485,Dwārka,IN,22.2394,68.9678,
9,490,Keti Bandar,PK,24.1447,67.4497,


In [107]:
# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Parameters dictionary for api call
parameters = {
    "radius": 5000,
    "keyword": "hotel",
    "key": google_key}

# Header of print log
print("Retrieving hotel names.")
print("----------------------")

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # Handle errors for cities with missing data
    try:
    
        # Change location of each iteration while leaving original parameters in place
        parameters["location"] = f"{row['Latitude']}, {row['Longitude']}"

        # Header for each API call
        print(f"Retrieving Results for Index {index}: {row['City']}.")

        # Assemble url and make API request
        response = requests.get(base_url, params = parameters).json()

        # Extract results
        results = response['results']
        
        # Print city name and closest hotel name
        print(f"In {row['City']}, the closest hotel is {results[0]['name']}.")
        
        # Add hotel name to relevant column in data frame
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        
        # Skip result if data is missing
        print("Missing field/result... skipping.")
        print("--------------------------------")
        pass
    
# Footer of print log
print("------------------------------")
print("Hotel name retrieval complete.")

hotel_df

Retrieving hotel names.
----------------------
Retrieving Results for Index 0: Ormara.
In Ormara, the closest hotel is Alqadri hotel zero point ormara.
Retrieving Results for Index 1: Sur.
In Sur, the closest hotel is Sur Plaza Hotel.
Retrieving Results for Index 2: Tura.
In Tura, the closest hotel is Hotel Polo Orchid.
Retrieving Results for Index 3: Pedernales.
In Pedernales, the closest hotel is Vista de Águilas Ecolodge.
Retrieving Results for Index 4: Kruisfontein.
In Kruisfontein, the closest hotel is Humansdorp Boutique Hotel.
Retrieving Results for Index 5: Plettenberg Bay.
In Plettenberg Bay, the closest hotel is Bayview Hotel.
Retrieving Results for Index 6: La Palma.
In La Palma, the closest hotel is Days Inn by Wyndham Buena Park.
Retrieving Results for Index 7: Chabahar.
In Chabahar, the closest hotel is Hotel sudos.
Retrieving Results for Index 8: Dwārka.
In Dwārka, the closest hotel is The Dwarika Hotel.
Retrieving Results for Index 9: Keti Bandar.
Missing field/result..

,index,City,Country,Latitude,Longitude,Hotel Name
0,102,Ormara,PK,25.2088,64.6357,Alqadri hotel zero point ormara
1,161,Sur,OM,22.5667,59.5289,Sur Plaza Hotel
2,224,Tura,IN,25.5198,90.2201,Hotel Polo Orchid
3,241,Pedernales,DO,18.0384,-71.7440,Vista de Águilas Ecolodge
4,299,Kruisfontein,ZA,-34.0033,24.7314,Humansdorp Boutique Hotel
5,384,Plettenberg Bay,ZA,-34.0527,23.3716,Bayview Hotel
6,427,La Palma,US,33.8464,-118.0467,Days Inn by Wyndham Buena Park
7,447,Chabahar,IR,25.2919,60.6430,Hotel sudos
8,485,Dwārka,IN,22.2394,68.9678,The Dwarika Hotel
9,490,Keti Bandar,PK,24.1447,67.4497,


In [108]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [113]:
# Create marker layer
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…